In [1]:
import numpy as np # import for math and array operations
import pandas as pd # import for dataframe handle
import matplotlib.pyplot as plt # import for visual representation
import seaborn as sns # import for visual representation
import math
from bs4 import BeautifulSoup
import requests
import datetime as dt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor


%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('../raw_data/combine_df.csv')

In [3]:
def clean_tags(tag):
    if ',' in str(tag):
        tag = ','.join(list(set(str(tag).split(','))))
        return tag
df['tags'] = df["popular_tags"].astype(str) +',' + df['genre'].astype(str)
df['tags'] = df['tags'].apply(lambda x : clean_tags(x))

In [4]:
df.head()

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,...,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price,clean_reviews,clean_date,tags
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...",...,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99,Very Positive,2016-05-12 00:00:00,"Atmospheric,Fast-Paced,Singleplayer,Remake,Mul..."
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...",...,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN,Mixed,2017-12-21 00:00:00,"Battle Royale,Multiplayer,PvP,Strategy,Tactica..."
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...",...,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN,Mostly Positive,2018-04-24 00:00:00,"Story Rich,Turn-Based Tactics,Singleplayer,Mul..."
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...",...,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN,Mixed,2018-12-13 00:00:00,"Atmospheric,Multiplayer,PvP,Military,Massively..."
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...",...,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN,Mostly Positive,2003-05-06 00:00:00,"Atmospheric,MMORPG,Free to Play,Multiplayer,Pv..."


In [5]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from unidecode import unidecode
import string

def clean (text):
    
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
        
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('English')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    return " ".join(without_stopwords)

In [6]:
df['clean_tag'] = df['tags'].apply(clean)

In [7]:
df = df.drop(['tags','genre' , 'popular_tags'],axis  =1)

In [8]:

# Tuned TFidfvectorizer
vec = TfidfVectorizer(min_df = 0.01).fit(df.clean_tag)
vectors = vec.transform(df.clean_tag) # Transform text to vectors


In [9]:
X_proj = pd.DataFrame(vectors.toarray(), index=df.name.tolist())

# Define X and y
y = X_proj[0]
knn_model = KNeighborsRegressor().fit(X_proj,y) 

In [271]:
X_proj.loc[['DOOM']]

,0,1,2,3,4,5,6,7,8,9,...,123,124,125,126,127,128,129,130,131,132
DOOM,0.0,0.09857,0.0,0.0,0.0,0.185869,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.269721


In [275]:
#Test
neighbors_list = list(knn_model.kneighbors(X_proj.loc[['QUAKE II']],n_neighbors=10)[1][0])
pd.DataFrame(knn_model.kneighbors(X_proj.loc[['QUAKE II']],n_neighbors=10)[0][0], index = X_proj.iloc[neighbors_list, :].index, columns = ['distance'])

,distance
QUAKE II,0.000000
Unreal Tournament: Game of the Year Edition,0.415224
QUAKE,0.459537
Unreal Tournament 3 Black,0.466273
Unreal Tournament 2004: Editor's Choice Edition,0.469410
Quake IV,0.520002
Deathmatch Classic,0.534551
Ultimate Doom,0.537888
Aliens versus Predator Classic 2000,0.574186
DOOM,0.583009


In [258]:
knn_model.kneighbors(X_proj.iloc[0:1],n_neighbors=10)[0][0]

array([1.49011612e-08, 5.83009146e-01, 5.87750793e-01, 6.19399838e-01,
       6.28136432e-01, 6.46344607e-01, 6.58609464e-01, 6.75810410e-01,
       6.77688194e-01, 7.01110496e-01])

In [243]:
knn_model

'DOOM'

### Testing

In [12]:
user_df= pd.read_csv('../raw_data/steam-200k.csv',usecols=[0,1,2,3],names=['userid','game','behavior','hoursplayed'])
user_df.head()

,userid,game,behavior,hoursplayed
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0


In [39]:
user_df['name']=user_df['game']
user_df = user_df[user_df['behavior']== 'play']

In [40]:
user_test= pd.DataFrame(user_df['name'].unique(),columns=['name'])
user_test

,name
0,The Elder Scrolls V Skyrim
1,Fallout 4
2,Spore
3,Fallout New Vegas
4,Left 4 Dead 2
...,...
3595,Space Colony
3596,Life is Hard
3597,Executive Assault
3598,O.R.B.


In [41]:
join_df=df.merge(user_test, on='name')
join_name = list(join_df.name.unique())

In [43]:
len(join_name)

1513

In [131]:
test_df = user_df[user_df['game'].isin(join_name)]

In [127]:
test_df = test_df.drop(columns = ['game', 'behavior'])

In [132]:
test_df

,userid,game,behavior,hoursplayed,name
3,151603712,Fallout 4,play,87.0,Fallout 4
9,151603712,Left 4 Dead 2,play,8.9,Left 4 Dead 2
11,151603712,HuniePop,play,8.5,HuniePop
13,151603712,Path of Exile,play,8.1,Path of Exile
15,151603712,Poly Bridge,play,7.5,Poly Bridge
...,...,...,...,...,...
199987,128470551,Rogue Legacy,play,2.6,Rogue Legacy
199989,128470551,Mortal Kombat Komplete Edition,play,2.5,Mortal Kombat Komplete Edition
199993,128470551,Magic Duels,play,2.2,Magic Duels
199995,128470551,Titan Souls,play,1.5,Titan Souls


In [152]:
test_df.groupby(['userid', 'name']).sum().sort_values(['userid','hoursplayed'], ascending = [True, False]).reset_index().groupby(['userid']).first()

,name,hoursplayed
userid,,
5250,Portal 2,13.6
76767,Counter-Strike,365.0
86540,Far Cry 3,17.8
144736,Counter-Strike,0.1
181212,Counter-Strike,1.8
...,...,...
309265377,Brawlhalla,2.0
309404240,Unturned,13.0
309434439,Dota 2,0.8


In [218]:
test_df.groupby(['userid', 'name']).sum().sort_values(['userid','hoursplayed'], ascending = [True, False])

hoursplayed
userid    name                             
5250      Portal 2                     13.6
          Alien Swarm                   4.9
          Team Fortress 2               0.8
          Dota 2                        0.2
76767     Counter-Strike              365.0
...                                     ...
309404240 AdVenture Capitalist          0.7
          Transformice                  0.3
309434439 Dota 2                        0.8
309824202 Dota 2                        0.7
309903146 Dota 2                        0.2

[36161 rows x 1 columns]

In [220]:
test_df[test_df['hoursplayed'] > 1]

,userid,game,behavior,hoursplayed,name
3,151603712,Fallout 4,play,87.0,Fallout 4
9,151603712,Left 4 Dead 2,play,8.9,Left 4 Dead 2
11,151603712,HuniePop,play,8.5,HuniePop
13,151603712,Path of Exile,play,8.1,Path of Exile
15,151603712,Poly Bridge,play,7.5,Poly Bridge
...,...,...,...,...,...
199987,128470551,Rogue Legacy,play,2.6,Rogue Legacy
199989,128470551,Mortal Kombat Komplete Edition,play,2.5,Mortal Kombat Komplete Edition
199993,128470551,Magic Duels,play,2.2,Magic Duels
199995,128470551,Titan Souls,play,1.5,Titan Souls


In [210]:
count_df

hoursplayed
userid    game                        
5250      Alien Swarm              4.9
          Dota 2                   0.2
          Portal 2                13.6
          Team Fortress 2          0.8
76767     Alien Swarm              0.8
...                                ...
309404240 Transformice             0.3
          Unturned                13.0
309434439 Dota 2                   0.8
309824202 Dota 2                   0.7
309903146 Dota 2                   0.2

[36161 rows x 1 columns]

In [205]:
list_user_more_hour = list(count_df[count_df['hoursplayed'] > 1].index)

In [217]:
len(list(set(count_df[count_df['hoursplayed'] > 1].reset_index().userid)))

7551

In [206]:
sum_df = test_df[test_df['userid'].isin(list_user_more_hour) == True].groupby('userid').count()[['game']]

In [207]:
sum_df

,game
userid,


In [183]:
list_user_more_game = list(sum_df[sum_df['game'] > 1].index)

In [197]:
final_test_df = test_df[test_df['userid'].isin(list_user_more_game) == True]

In [231]:
game_df = test_df.groupby('userid').count()[['game']]

In [235]:
list(game_df[game_df['game'] > 1].index)

[5250,
 76767,
 86540,
 229911,
 298950,
 554278,
 561758,
 622362,
 748719,
 861238,
 871990,
 948368,
 975449,
 994489,
 1072465,
 1129452,
 1364546,
 1612666,
 1936551,
 1950243,
 2039434,
 2083767,
 2110581,
 2184507,
 2259650,
 2428602,
 2531540,
 2643609,
 2753525,
 3449240,
 3450426,
 3527485,
 3783783,
 3979930,
 4064412,
 4595423,
 4812175,
 4824107,
 4834220,
 5232588,
 5270060,
 5387470,
 5860071,
 5949488,
 5976642,
 5990132,
 6144819,
 6252885,
 6439568,
 6717871,
 6928806,
 7015471,
 7163917,
 7203603,
 7249363,
 7418489,
 7431946,
 7440594,
 7907686,
 7955670,
 8259307,
 8311704,
 8542204,
 8585433,
 8776918,
 8784496,
 8795607,
 8949216,
 9065641,
 9128105,
 9180659,
 9209946,
 9245216,
 9365873,
 9563573,
 9740704,
 9823354,
 9845449,
 9946133,
 10162301,
 10181060,
 10218563,
 10253354,
 10302000,
 10385370,
 10450544,
 10478285,
 10599862,
 11024186,
 11149819,
 11161178,
 11207650,
 11300897,
 11373749,
 11403772,
 11731710,
 11794760,
 11807754,
 11813637,
 1194033

In [202]:
final_test_df.groupby(['userid','game']).sum().sort_values('hoursplayed')

,,hoursplayed
userid,game,
54888069,Team Fortress Classic,0.1
235996643,Two Worlds Epic Edition,0.1
96400571,Banished,0.1
31187179,Rock of Ages,0.1
80779496,LIMBO,0.1
...,...,...
12660489,Dota 2,5970.0
86256882,Dota 2,6015.0
121199670,Dota 2,6753.0
